# Main imports and code

In [ ]:
# check which gpu we're using
!nvidia-smi

Sat Jan  8 17:59:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip  install transformers
!pip install pytorch-ignite

     |████████████████████████████████| 3.4 MB 12.4 MB/s 
     |████████████████████████████████| 3.3 MB 39.7 MB/s 
     |████████████████████████████████| 895 kB 49.8 MB/s 
     |████████████████████████████████| 596 kB 52.0 MB/s 
     |████████████████████████████████| 61 kB 433 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 240 kB 12.1 MB/s 


In [ ]:
# Any results you write to the current directory are saved as output.
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from transformers import BertTokenizer,BertModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
from torch.nn.utils.rnn import pack_padded_sequence
from torch.optim import AdamW
from tqdm import tqdm
from argparse import ArgumentParser
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.engine.engine import Engine, State, Events
from ignite.handlers import EarlyStopping
from ignite.contrib.handlers import TensorboardLogger, ProgressBar
from ignite.utils import convert_tensor
from torch.optim.lr_scheduler import ExponentialLR
import warnings  
warnings.filterwarnings('ignore')

In [ ]:
import os
import gc
import copy
import time
import random
import string

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

# Utils
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

In [ ]:
from transformers import AutoTokenizer, AutoModel, AdamW


In [ ]:
!pip install sentencepiece


     |████████████████████████████████| 1.2 MB 12.9 MB/s 


In [ ]:
import random
import os
from urllib import request

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/ISarcasm/DataSet/train.En.csv')
df=df[['tweet','sarcastic']]

In [ ]:
train, validate, test = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.6*len(df)), int(.8*len(df))])

In [ ]:
train=pd.concat([train, validate], ignore_index=True)

In [ ]:
# tedf1.to_csv('/content/drive/MyDrive/PCL/test_task_1',index=False)

In [ ]:
# trdf1.to_csv('/content/drive/MyDrive/PCL/train_task_1',index=False)

# FE extraction

In [ ]:
def count_character_type(text):
    text=str(text)

    specialChar = 0
    for i in range(0, len(text)):
        ch = text[i]
        if ch == "!":
            specialChar+=1
        
    return specialChar
    
def count_question_mark(text):
    text=str(text)
    specialChar = 0
    for i in range(0, len(text)):
        ch = text[i]
        if ch == "?":
            specialChar+=1
        
    return specialChar

The Python NLP library, TextBlob has a method to roughly quantify if a sentence is fact or opinion.¶
The method outputs a number ranging from 0 to 1, an output close to 0 indicates the sentence is highly factual and close to 1 means the sentence is highly subjective. Here, we take the sum for every comment, an overall higher sum then, will be indicative of higher subjectivity

In [ ]:
def get_subjectivity(text):
    return TextBlob(str(text)).sentiment.subjectivity

The distribution of the 4 most important parts-of-speech (Noun, Verb, Adjective,Pronoun) in sarcastic and non sarcastic comments might shed some light, so let's plot it as a grouped bar chart

In [ ]:
import spacy

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nlp = spacy.load('en_core_web_sm')
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
def pos_counter(doc):
    verb_count = 0
    adj_count = 0
    pron_count = 0
    noun_count=0
    for tok in doc:
        if tok.pos_ == "VERB":
            verb_count=verb_count+1
        elif tok.pos_ == "ADJ":
            adj_count=adj_count+1
        elif tok.pos_ == "PRON":
            pron_count=pron_count+1
        elif tok.pos_ == "PROPN":
            noun_count=noun_count+1
    return (verb_count,adj_count,pron_count,noun_count)

In [ ]:
def get_verb_count(text):
  verb_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "VERB":
            verb_count=verb_count+1
  return verb_count

def get_noun_count(text):
  noun_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "PROPN":
            noun_count=noun_count+1
  return noun_count

def get_pron_count(text):
  pron_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "PRON":
            pron_count=pron_count+1
  return pron_count


def get_adj_count(text):
  adj_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "ADJ":
            adj_count=adj_count+1
  return adj_count

Sarcastic comments tend to have profanity interspersed within, let's get a count of both sarcastic and non sarcastic and see how they compare

In [ ]:
!pip install better_profanity 
from better_profanity import profanity

     |████████████████████████████████| 46 kB 2.8 MB/s 


In [ ]:
def count_profane_words(text):
    count = 0
    for sent in str(text).split():
        if profanity.contains_profanity(sent) == True:
            count = count+1
    return count

get entity cloud

In [ ]:
def entity_wordcloud(text):
    named_entities = []    
    sent = nlp(str(text))
    for ent in sent.ents:
        if ent.label_ == 'PERSON' or 'ORG' or 'GPE':
            named_entities.append(ent.text)
            
    doc = " ".join(named_entities)
    return doc

In [ ]:
!git clone https://github.com/larsmans/sentiwordnet.git

Cloning into 'sentiwordnet'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 0), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [ ]:
import csv

In [ ]:
class load_senti_word_net(object):
    """
    constructor to load the file and read the file as CSV
    6 columns - pos, ID, PosScore, NegScore, synsetTerms, gloss
    synsetTerms can have multiple similar words like abducting#1 abducent#1 and will read each one and calculaye the scores
    """
    
    def __init__(self):
        sent_scores = collections.defaultdict(list)
        with io.open("/content/sentiwordnet/SentiWordNet_3.0.0_20130122.txt") as fname:
            file_content = csv.reader(fname, delimiter='\t',quotechar='"')
            
            for line in file_content:                
                if line[0].startswith('#') :
                    continue                    
                pos, ID, PosScore, NegScore, synsetTerms, gloss = line
                for terms in synsetTerms.split(" "):
                    term = terms.split("#")[0]
                    term = term.replace("-","").replace("_","")
                    key = "%s/%s"%(pos,term.split("#")[0])
                    try:
                        sent_scores[key].append((float(PosScore),float(NegScore)))
                    except:
                        sent_scores[key].append((0,0))
                    
        for key, value in sent_scores.items():
            sent_scores[key] = np.mean(value,axis=0)
        
        self.sent_scores = sent_scores    
     
    """
    For a word,
    nltk.pos_tag(["Suraj"])
    [('Suraj', 'NN')]
    """
    
    def score_word(self, word):
        pos = nltk.pos_tag([word])[0][1]
        return self.score(word, pos)
    
    def score(self,word, pos):
        """
        Identify the type of POS, get the score from the senti_scores and return the score
        """
        
        if pos[0:2] == 'NN':
            pos_type = 'n'
        elif pos[0:2] == 'JJ':
            pos_type = 'a'
        elif pos[0:2] =='VB':
            pos_type='v'
        elif pos[0:2] =='RB':
            pos_type = 'r'
        else:
            pos_type =  0
            
        if pos_type != 0 :    
            loc = pos_type+'/'+word
            score = self.sent_scores[loc]
            if len(score)>1:
                return score
            else:
                return np.array([0.0,0.0])
        else:
            return np.array([0.0,0.0])
        
    """
    Repeat the same for a sentence
    nltk.pos_tag(word_tokenize("My name is Suraj"))
    [('My', 'PRP$'), ('name', 'NN'), ('is', 'VBZ'), ('Suraj', 'NNP')]    
    """    
        
    def score_sentencce(self, sentence):
        pos = nltk.pos_tag(sentence)
        # print (pos)
        mean_score = np.array([0.0, 0.0])
        for i in range(len(pos)):
            mean_score += self.score(pos[i][0], pos[i][1])
            
        return mean_score
    
    def pos_vector(self, sentence):
        pos_tag = nltk.pos_tag(sentence)
        vector = np.zeros(4)
        
        for i in range(0, len(pos_tag)):
            pos = pos_tag[i][1]
            if pos[0:2]=='NN':
                vector[0] += 1
            elif pos[0:2] =='JJ':
                vector[1] += 1
            elif pos[0:2] =='VB':
                vector[2] += 1
            elif pos[0:2] == 'RB':
                vector[3] += 1
                
        return vector
            

In [ ]:
def gram_features(features,sentence):
    sentence_rep = replace_reg(str(sentence))
    token = nltk.word_tokenize(sentence_rep)
    token = [porter.stem(i.lower()) for i in token]        
    
    bigrams = nltk.bigrams(token)
    bigrams = [tup[0] + ' ' + tup[1] for tup in bigrams]
    grams = token + bigrams
    #print (grams)
    for t in grams:
        features['contains(%s)'%t]=1.0

In [ ]:
def pos_features(features,sentence):
    sentence_rep = replace_reg(sentence)
    token = nltk.word_tokenize(sentence_rep)
    token = [ porter.stem(each.lower()) for each in token]
    pos_vector = sentiments.pos_vector(token)
    for j in range(len(pos_vector)):
        features['POS_'+str(j+1)] = pos_vector[j]
    # print ("done")
    

In [ ]:
def capitalization(features,sentence):
    count = 0
    for i in range(len(sentence)):
        count += int(sentence[i].isupper())
    features['Capitalization'] = int(count > 3)
    # print (count)

In [ ]:
import nltk
import re

#dictionnary to sentiment analysis
emo_repl = {
    #good emotions
    "&lt;3" : " good ",
    ":d" : " good ",
    ":dd" : " good ",
    ":p" : " good ",
    "8)" : " good ",
    ":-)" : " good ",
    ":)" : " good ",
    ";)" : " good ",
    "(-:" : " good ",
    "(:" : " good ",
    
    "yay!" : " good ",
    "yay" : " good ",
    "yaay" : " good ",
    "yaaay" : " good ",
    "yaaaay" : " good ",
    "yaaaaay" : " good ",    
    #bad emotions
    ":/" : " bad ",
    ":&gt;" : " sad ",
    ":')" : " sad ",
    ":-(" : " bad ",
    ":(" : " bad ",
    ":s" : " bad ",
    ":-s" : " bad "
}

#dictionnary for general (i.e. topic modeler)
emo_repl2 = {
    #good emotions
    "&lt;3" : " heart ",
    ":d" : " smile ",
    ":p" : " smile ",
    ":dd" : " smile ",
    "8)" : " smile ",
    ":-)" : " smile ",
    ":)" : " smile ",
    ";)" : " smile ",
    "(-:" : " smile ",
    "(:" : " smile ",
       
    #bad emotions
    ":/" : " worry ",
    ":&gt;" : " angry ",
    ":')" : " sad ",
    ":-(" : " sad ",
    ":(" : " sad ",
    ":s" : " sad ",
    ":-s" : " sad "
}

#general
re_repl = {
    r"\br\b" : "are",
    r"\bu\b" : "you",
    r"\bhaha\b" : "ha",
    r"\bhahaha\b" : "ha",
    r"\bdon't\b" : "do not",
    r"\bdoesn't\b" : "does not",
    r"\bdidn't\b" : "did not",
    r"\bhasn't\b" : "has not",
    r"\bhaven't\b" : "have not",
    r"\bhadn't\b" : "had not",
    r"\bwon't\b" : "will not",
    r"\bwouldn't\b" : "would not",
    r"\bcan't\b" : "can not",
    r"\bcannot\b" : "can not"    
}

emo_repl_order = [k for (k_len,k) in reversed(sorted([(len(k),k) for k in emo_repl.keys()]))]
emo_repl_order2 = [k for (k_len,k) in reversed(sorted([(len(k),k) for k in emo_repl2.keys()]))]

def replace_emo(sentence):
    sentence2 = sentence
    for k in emo_repl_order:
        sentence2 = sentence2.replace(k,emo_repl[k])
    for r, repl in re_repl.items():
        sentence2 = re.sub(r,repl,sentence2)
    return sentence2

def replace_reg(sentence):
    sentence2 = sentence
    for k in emo_repl_order2:
        sentence2 = sentence2.replace(k,emo_repl2[k])
    for r, repl in re_repl.items(): 
        sentence2 = re.sub(r,repl,sentence2)
    return sentence2

In [ ]:
from gensim import corpora, models, similarities
import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd

class topic(object):
    def __init__(self, nbtopic = 100, alpha=1,model=None,dicttp=None):
        self.nbtopic = nbtopic
        self.alpha = alpha
        self.porter = nltk.PorterStemmer()
        self.stop = stopwords.words('english')+['.','!','?','"','...','\\',"''",'[',']','~',"'m","'s",';',':','..','$']
        if model!=None and dicttp!=None:
            self.lda = models.ldamodel.LdaModel.load(model)
            self.dictionary =  corpora.Dictionary.load(dicttp)
            
    def fit(self,documents):
        
        documents_mod = documents
        tokens = [nltk.word_tokenize(sentence) for sentence in documents_mod]
        tokens = [[self.porter.stem(t.lower()) for t in sentence if t.lower() not in self.stop] for sentence in tokens]        
            
        self.dictionary = corpora.Dictionary(tokens)
        corpus = [self.dictionary.doc2bow(text) for text in tokens]
        self.lda = models.ldamodel.LdaModel(corpus,id2word=self.dictionary, num_topics=self.nbtopic,alpha=self.alpha)
        
        self.lda.save('topics.tp')
        self.dictionary.save('topics_dict.tp')
        
    def get_topic(self,topic_number):
        
        return self.lda.print_topic(topic_number)
    
    def transform(self,sentence):
        
        sentence_mod = sentence
        tokens = nltk.word_tokenize(sentence_mod)
        tokens = [self.porter.stem(t.lower()) for t in tokens if t.lower() not in self.stop] 
        corpus_sentence = self.dictionary.doc2bow(tokens)
        
        return self.lda[corpus_sentence] 

In [ ]:
porter = nltk.PorterStemmer()


In [ ]:
import string
def sentiment_extract(features, sentence):
    sentence_rep = replace_reg(sentence)
    token = nltk.word_tokenize(sentence_rep)    
    token = [porter.stem(i.lower()) for i in token]   
    mean_sentiment = sentiments.score_sentencce(token)
    features["Positive Sentiment"] = mean_sentiment[0]
    features["Negative Sentiment"] = mean_sentiment[1]
    features["sentiment"] = mean_sentiment[0] - mean_sentiment[1]
    #print(mean_sentiment[0], mean_sentiment[1])
    
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in token]).strip())
        features["Blob Polarity"] = text.sentiment.polarity
        features["Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["Blob Polarity"] = 0
        features["Blob Subjectivity"] = 0
        # print("do nothing")
        
    
    first_half = token[0:len(token)//2]    
    mean_sentiment_half = sentiments.score_sentencce(first_half)
    features["positive Sentiment first half"] = mean_sentiment_half[0]
    features["negative Sentiment first half"] = mean_sentiment_half[1]
    features["first half sentiment"] = mean_sentiment_half[0]-mean_sentiment_half[1]
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in first_half]).strip())
        features["first half Blob Polarity"] = text.sentiment.polarity
        features["first half Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["first Blob Polarity"] = 0
        features["first Blob Subjectivity"] = 0
        # print("do nothing")
    
    second_half = token[len(token)//2:]
    mean_sentiment_sechalf = sentiments.score_sentencce(second_half)
    features["positive Sentiment second half"] = mean_sentiment_sechalf[0]
    features["negative Sentiment second half"] = mean_sentiment_sechalf[1]
    features["second half sentiment"] = mean_sentiment_sechalf[0]-mean_sentiment_sechalf[1]
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in second_half]).strip())
        features["second half Blob Polarity"] = text.sentiment.polarity
        features["second half Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["second Blob Polarity"] = 0
        features["second Blob Subjectivity"] = 0
        # print("do nothing")  
    

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
topic_mod = topic(nbtopic=200,alpha='symmetric')
topic_mod.fit(train['tweet'].values)
# topic_mod = topic(model=os.path.join('topics.tp'),dicttp=os.path.join('topics_dict.tp'))
def topic_feature(features,sentence,topic_modeler):    
    topics = topic_modeler.transform(sentence)    
    for j in range(len(topics)):
        features['Topic :'] = topics[j][1]

In [ ]:
!pip install emoji

     |████████████████████████████████| 170 kB 12.1 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=1dbf709a52e00cbefe5a727ae875830bdfab9f657b86e7c00b7cefdb77433f95
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [ ]:
from emoji import UNICODE_EMOJI



In [ ]:
"""
Function to extract the below features.
1. Presence of emoji's
2. Count of number of Question marks
3. Count of number of Exclamation marks
4. presence of hashtags other than #sarcasm
5. presence of any @user tweets

"""
from collections import defaultdict

def Emoji_present(text):
    emoji = {
    "&lt;3" : "positive",":D" : "positive",	":d" : "positive", ":dd" : "positive", ":P" : "positive", ":p" : "positive","8)" : "positive",
    "8-)" : "positive",  ":-)" : "positive",    ":)" : "positive",    ";)" : "positive",    "(-:" : "positive",    "(:" : "positive",
    ":')" : "positive",    "xD" : "positive",    "XD" : "positive",  "yay!" : "positive",  "yay" : "positive",  "yaay" : "positive",
    "yaaay" : "positive",  "yaaaay" : "positive", "yaaaaay" : "positive", "Yay!" : "positive", "Yay" : "positive", "Yaay" : "positive",
    "Yaaay" : "positive", "Yaaaay" : "positive", "Yaaaaay" : "positive",  ":/" : "negative", "&gt;" : "negative", ":'(" : "negative",
    ":-(" : "negative", ":(" : "negative", ":s" : "negative",":-s" : "negative","-_-" : "negative", "-.-" : "negative" }
    emoji_count =defaultdict(int) 
    isPresent=False    
        
    for word in str(text):
        if word in emoji:            
            isPresent = True

    
    count = 0
    for emoji in UNICODE_EMOJI:
        count += str(text).count(emoji)
        if count>=1:
          isPresent=True      

    
    return isPresent 



In [ ]:
import collections


In [ ]:
import io

sentiments = load_senti_word_net()

def get_features(sentence, topic_modeler):
    features = {}
    # gram_features(features,sentence)
    pos_features(features,sentence)
    sentiment_extract(features, sentence)
    capitalization(features,sentence)
    topic_feature(features, sentence,topic_modeler)
    return features

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
train.dropna(inplace=True)

In [ ]:
train

,tweet,sarcastic
0,Less than 5 minutes after getting my phone bac...,0
1,Love seeing 15 year olds being vaccinated in t...,1
2,I get a lot of boy who cried wolf vibes from t...,1
3,im about to just walk into a place and start w...,1
4,Every time I comment on a TikTok asking for ca...,0
...,...,...
2769,This weekend I hit a delivery biker with a car...,0
2770,This country is screwed let’s be honest,0
2771,I always come up with some pretty interesting ...,0
2772,@andy_mansell72 @VicOtley @garry_whu @NiaAnts ...,1


In [ ]:
import re
featureset=[]
for i in range(0,len(train)):
    # if i==2774or i==2775:
    #   continue
    temp = str(train["tweet"][i])
    temp = re.sub(r'[^\x00-\x7F]+','',temp)
    featureset.append((get_features(temp,topic_mod), train["sarcastic"][i], train["tweet"][i]))

In [ ]:
c = []
for i in range(0,len(featureset)):
    c.append(pd.DataFrame(featureset[i][0],index=[i]))
result = pd.concat(c)

In [ ]:
result.insert(loc=0,column="label",value='0')
for i in range(0, len(featureset)):
    result["label"].loc[i] = featureset[i][1]   

result.insert(loc=0,column="text",value='0')
for i in range(0, len(featureset)):
    result["text"].loc[i] = featureset[i][2]  
    

In [ ]:
result.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first Blob Polarity,first Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second Blob Polarity,second Blob Subjectivity,Capitalization,Topic :
0,Less than 5 minutes after getting my phone bac...,0,11.0,2.0,2.0,1.0,0.095620,0.432559,-0.336939,0,0,0.076389,0.315972,-0.239583,0,0,0.019231,0.116587,-0.097356,0,0,0,0.786919
1,Love seeing 15 year olds being vaccinated in t...,1,7.0,2.0,6.0,0.0,0.980875,0.337487,0.643387,0,0,0.856971,0.159856,0.697115,0,0,0.123904,0.177632,-0.053728,0,0,0,0.118817
2,I get a lot of boy who cried wolf vibes from t...,1,8.0,2.0,3.0,0.0,0.380556,0.743651,-0.363095,0,0,0.097222,0.251984,-0.154762,0,0,0.283333,0.491667,-0.208333,0,0,0,0.072668
3,im about to just walk into a place and start w...,1,6.0,0.0,3.0,3.0,0.213412,0.136337,0.077076,0,0,0.157961,0.039062,0.118899,0,0,0.055451,0.097274,-0.041823,0,0,0,0.083048
4,Every time I comment on a TikTok asking for ca...,0,15.0,3.0,1.0,0.0,0.717014,0.522222,0.194792,0,0,0.458333,0.466667,-0.008333,0,0,0.258681,0.055556,0.203125,0,0,1,0.078510


In [ ]:
result['contain_emoji']=result['text'].apply(Emoji_present)

In [ ]:
result['count_special_chracter']=result['text'].apply(count_character_type)

In [ ]:
result['count_question_marks']=result['text'].apply(count_question_mark)

In [ ]:
from textblob import TextBlob

In [ ]:
result['subjectivity']=result['text'].apply(get_subjectivity)

In [ ]:
result['count_verbs']=result['text'].apply(get_verb_count)

In [ ]:
result['count_nouns']=result['text'].apply(get_noun_count)

In [ ]:
result['count_pronun']=result['text'].apply(get_pron_count)

In [ ]:
result['count_adjct']=result['text'].apply(get_adj_count)

In [ ]:
result['entities']=result['text'].apply(entity_wordcloud)

In [ ]:
result['count_profane']=result['text'].apply(count_profane_words)

In [ ]:
result.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first Blob Polarity,first Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second Blob Polarity,second Blob Subjectivity,Capitalization,Topic :,contain_emoji,count_special_chracter,count_question_marks,subjectivity,count_verbs,count_nouns,count_pronun,count_adjct,entities,count_profane
0,Less than 5 minutes after getting my phone bac...,0,11.0,2.0,2.0,1.0,0.095620,0.432559,-0.336939,0,0,0.076389,0.315972,-0.239583,0,0,0.019231,0.116587,-0.097356,0,0,0,0.786919,True,0,0,0.033333,2,0,1,1,Less than 5 minutes,0
1,Love seeing 15 year olds being vaccinated in t...,1,7.0,2.0,6.0,0.0,0.980875,0.337487,0.643387,0,0,0.856971,0.159856,0.697115,0,0,0.123904,0.177632,-0.053728,0,0,0,0.118817,True,0,0,0.466667,3,1,1,1,15 year olds the end of June first,0
2,I get a lot of boy who cried wolf vibes from t...,1,8.0,2.0,3.0,0.0,0.380556,0.743651,-0.363095,0,0,0.097222,0.251984,-0.154762,0,0,0.283333,0.491667,-0.208333,0,0,0,0.072668,True,0,0,0.000000,4,2,2,0,wolf vibes the Red Cross,0
3,im about to just walk into a place and start w...,1,6.0,0.0,3.0,3.0,0.213412,0.136337,0.077076,0,0,0.157961,0.039062,0.118899,0,0,0.055451,0.097274,-0.041823,0,0,0,0.083048,True,0,0,0.000000,6,0,3,0,,0
4,Every time I comment on a TikTok asking for ca...,0,15.0,3.0,1.0,0.0,0.717014,0.522222,0.194792,0,0,0.458333,0.466667,-0.008333,0,0,0.258681,0.055556,0.203125,0,0,1,0.078510,True,0,0,0.533333,6,4,3,2,TikTok,0


In [ ]:
len(test)

694

In [ ]:
import re
feature_set_test=[]
for i, row in test.iterrows():
    # if i==0:
    #   continue
    temp = str(test["tweet"][i])
    temp = re.sub(r'[^\x00-\x7F]+','',temp)
    feature_set_test.append((get_features(temp,topic_mod), test["sarcastic"][i],test['tweet'][i]))

In [ ]:
c = []
for i in range(0,len(feature_set_test)):
    c.append(pd.DataFrame(feature_set_test[i][0],index=[i]))
result_test = pd.concat(c)

In [ ]:
result_test.insert(loc=0,column="label",value='0')
for i in range(0, len(feature_set_test)):
    result_test["label"].loc[i] = feature_set_test[i][1]   

result_test.insert(loc=0,column="text",value='0')
for i in range(0, len(feature_set_test)):
    result_test["text"].loc[i] = feature_set_test[i][2]   
    
    

In [ ]:
result_test.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :
0,The world according to Jeff Goldblum is such a...,0,4.0,1.0,2.0,0.0,0.322917,0.354167,-0.031250,0.000000,0.500000,0.156250,0.187500,-0.031250,0.00,0.0,0.166667,0.166667,0.000000,0.000000,0.500000,0,0.251250
1,@kevinabstract ye,0,2.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.367771
2,Rubbing my tears into my face is part of my sk...,1,6.0,0.0,2.0,0.0,0.019231,0.184295,-0.165064,0.800000,0.700000,0.019231,0.184295,-0.165064,0.00,0.0,0.000000,0.000000,0.000000,0.800000,0.700000,0,0.143572
3,What's the German word for when you start a po...,0,10.0,5.0,7.0,3.0,0.922618,0.699750,0.222868,0.134091,0.188636,0.216518,0.158482,0.058036,0.05,0.1,0.706100,0.541268,0.164833,0.218182,0.277273,0,0.093709
4,"Prayers go up, blessings come down",0,2.0,0.0,2.0,0.0,0.077381,0.010119,0.067262,-0.155556,0.288889,0.041667,0.004167,0.037500,0.00,0.0,0.035714,0.005952,0.029762,-0.155556,0.288889,0,0.200284


In [ ]:
result_test['contain_emoji']=result_test['text'].apply(Emoji_present)

In [ ]:
result_test['count_special_chracter']=result_test['text'].apply(count_character_type)

In [ ]:
result_test['count_question_marks']=result_test['text'].apply(count_question_mark)

In [ ]:
result_test['subjectivity']=result_test['text'].apply(get_subjectivity)

In [ ]:
result_test['count_verbs']=result_test['text'].apply(get_verb_count)

In [ ]:
result_test['count_nouns']=result_test['text'].apply(get_noun_count)

In [ ]:
result_test['count_pronun']=result_test['text'].apply(get_pron_count)

In [ ]:
result_test['count_adjct']=result_test['text'].apply(get_adj_count)

In [ ]:
result_test['entities']=result_test['text'].apply(entity_wordcloud)

In [ ]:
result_test['count_profane']=result_test['text'].apply(count_profane_words)

In [ ]:
result_test.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :,contain_emoji,count_special_chracter,count_question_marks,subjectivity,count_verbs,count_nouns,count_pronun,count_adjct,entities,count_profane
0,The world according to Jeff Goldblum is such a...,0,4.0,1.0,2.0,0.0,0.322917,0.354167,-0.031250,0.000000,0.500000,0.156250,0.187500,-0.031250,0.00,0.0,0.166667,0.166667,0.000000,0.000000,0.500000,0,0.251250,False,0,0,0.500000,1,2,0,0,Jeff Goldblum,0
1,@kevinabstract ye,0,2.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.367771,False,0,0,0.000000,0,2,0,0,,0
2,Rubbing my tears into my face is part of my sk...,1,6.0,0.0,2.0,0.0,0.019231,0.184295,-0.165064,0.800000,0.700000,0.019231,0.184295,-0.165064,0.00,0.0,0.000000,0.000000,0.000000,0.800000,0.700000,0,0.143572,False,0,0,0.700000,1,0,0,0,,0
3,What's the German word for when you start a po...,0,10.0,5.0,7.0,3.0,0.922618,0.699750,0.222868,0.134091,0.188636,0.216518,0.158482,0.058036,0.05,0.1,0.706100,0.541268,0.164833,0.218182,0.277273,0,0.093709,True,0,1,0.188636,4,0,5,4,German 2 years old present-day every week 3 mo...,0
4,"Prayers go up, blessings come down",0,2.0,0.0,2.0,0.0,0.077381,0.010119,0.067262,-0.155556,0.288889,0.041667,0.004167,0.037500,0.00,0.0,0.035714,0.005952,0.029762,-0.155556,0.288889,0,0.200284,True,0,0,0.288889,2,0,0,0,,0


In [ ]:
result_test.columns

Index(['text', 'label', 'POS_1', 'POS_2', 'POS_3', 'POS_4',
       'Positive Sentiment', 'Negative Sentiment', 'sentiment',
       'Blob Polarity', 'Blob Subjectivity', 'positive Sentiment first half',
       'negative Sentiment first half', 'first half sentiment',
       'first half Blob Polarity', 'first half Blob Subjectivity',
       'positive Sentiment second half', 'negative Sentiment second half',
       'second half sentiment', 'second half Blob Polarity',
       'second half Blob Subjectivity', 'Capitalization', 'Topic :',
       'contain_emoji', 'count_special_chracter', 'count_question_marks',
       'subjectivity', 'count_verbs', 'count_nouns', 'count_pronun',
       'count_adjct', 'entities', 'count_profane'],
      dtype='object')

In [ ]:
result_test.dropna(inplace=True)

In [ ]:
result.dropna(inplace=True)

In [ ]:
x_train,y_train,x_train_text=result[['POS_1', 'POS_2', 'POS_3', 'POS_4',
       'Positive Sentiment', 'Negative Sentiment', 'sentiment',
       'Blob Polarity', 'Blob Subjectivity', 'positive Sentiment first half',
       'negative Sentiment first half', 'first half sentiment',
       'first half Blob Polarity', 'first half Blob Subjectivity',
       'positive Sentiment second half', 'negative Sentiment second half',
       'second half sentiment', 'second half Blob Polarity',
       'second half Blob Subjectivity', 'Capitalization', 'Topic :',
       'contain_emoji', 'count_special_chracter', 'count_question_marks',
       'subjectivity', 'count_verbs', 'count_nouns', 'count_pronun',
       'count_adjct', 'count_profane']],result[['label']],result[['text']]

In [ ]:
x_test,y_test,x_test_text=result_test[['POS_1', 'POS_2', 'POS_3', 'POS_4',
       'Positive Sentiment', 'Negative Sentiment', 'sentiment',
       'Blob Polarity', 'Blob Subjectivity', 'positive Sentiment first half',
       'negative Sentiment first half', 'first half sentiment',
       'first half Blob Polarity', 'first half Blob Subjectivity',
       'positive Sentiment second half', 'negative Sentiment second half',
       'second half sentiment', 'second half Blob Polarity',
       'second half Blob Subjectivity', 'Capitalization', 'Topic :',
       'contain_emoji', 'count_special_chracter', 'count_question_marks',
       'subjectivity', 'count_verbs', 'count_nouns', 'count_pronun',
       'count_adjct', 'count_profane']],result_test[['label']],result_test[['text']]

# RoBERTa Baseline for Task 1

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
!pip install datasets


     |████████████████████████████████| 306 kB 12.2 MB/s 
     |████████████████████████████████| 1.1 MB 47.3 MB/s 
     |████████████████████████████████| 243 kB 51.5 MB/s 
     |████████████████████████████████| 132 kB 51.5 MB/s 
     |████████████████████████████████| 192 kB 53.0 MB/s 
     |████████████████████████████████| 271 kB 55.3 MB/s 
     |████████████████████████████████| 160 kB 52.3 MB/s 


In [ ]:
class PCLTrainDataset(Dataset):
    def __init__(self, df, tokenizer, max_length,displacemnt):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.text = df['text'].values
        self.fetures = df[['POS_1', 'POS_2', 'POS_3', 'POS_4',
       'Positive Sentiment', 'Negative Sentiment', 'sentiment',
       'Blob Polarity', 'Blob Subjectivity', 'positive Sentiment first half',
       'negative Sentiment first half', 'first half sentiment',
       'first half Blob Polarity', 'first half Blob Subjectivity',
       'positive Sentiment second half', 'negative Sentiment second half',
       'second half sentiment', 'second half Blob Polarity',
       'second half Blob Subjectivity', 'Capitalization', 'Topic :',
       'contain_emoji', 'count_special_chracter', 'count_question_marks',
       'subjectivity', 'count_verbs', 'count_nouns', 'count_pronun',
       'count_adjct', 'count_profane']].astype(float).values
       
        self.label=df['label'].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        # summary = self.summary[index]
        inputs_text = self.tokenizer.encode_plus(
                                text,
                                truncation=True,
                                add_special_tokens=True,
                                max_length=self.max_len,
                                padding='max_length'
                            )
        
                            
        target = self.label[index]
        
        text_ids = inputs_text['input_ids']
        text_mask = inputs_text['attention_mask']
        
        
       
        
        
        return {
            
            'text_ids': torch.tensor(text_ids, dtype=torch.long),
            'text_mask': torch.tensor(text_mask, dtype=torch.long),
            'features': torch.tensor(self.fetures[index], dtype=torch.float),
            'target': torch.tensor(target, dtype=torch.float)
        }


In [ ]:
import math
def sigmoid(x):
    return 1/(1+math.exp(-x))

In [ ]:
class Recall_Loss(nn.Module):
    '''Calculate F1 score. Can work with gpu tensors
    
    The original implmentation is written by Michal Haltuf on Kaggle.
    
    Returns
    -------
    torch.Tensor
        `ndim` == 1. epsilon <= val <= 1
    
    Reference
    ---------
    - https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
    - https://discuss.pytorch.org/t/calculating-precision-recall-and-f1-score-in-case-of-multi-label-classification/28265/6
    - http://www.ryanzhang.info/python/writing-your-own-loss-function-module-for-pytorch/
    '''
    def __init__(self, epsilon=1e-7):
        super().__init__()
        self.epsilon = epsilon
        
    def forward(self, y_pred, y_true,):
        # assert y_pred.ndim == 2
        # assert y_true.ndim == 1
        # print(y_pred.shape)
        # print(y_true.shape)
        # y_pred[y_pred<0.5]=0
        # y_pred[y_pred>=0.5]=0


        
        y_true_one_hot = F.one_hot(y_true.to(torch.int64), 2).to(torch.float32)
        # y_pred_one_hot = F.one_hot(y_pred.to(torch.int64), 2).to(torch.float32)
        
        tp = (y_true_one_hot * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true_one_hot) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true_one_hot) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true_one_hot * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        # f1 = 2* (precision*recall) / (precision + recall + self.epsilon)
        # f1 = f1.clamp(min=self.epsilon, max=1-self.epsilon)
        # f1=f1.detach()
        # print(f1.shape)
        # y_pred=y_pred.reshape((y_pred.shape[0], 1))
        # y_true=y_true.reshape((y_true.shape[0], 1))

        # p1=y_true*(math.log(sigmoid(y_pred)))*(1-f1)[1]
        # p0=(1-y_true)*math.log(1-sigmoid(y_pred))*(1-f1)[0]


        # y_true_one_hot = F.one_hot(y_true.to(torch.int64), 2)
        # print(y_pred)
        # print(y_true_one_hot)
        recall=recall.detach()
        CE =torch.nn.CrossEntropyLoss(weight=( 1 - recall))(y_pred, y_true_one_hot)
        # loss = ( 1 - f1)  * CE
        return  CE.mean()

In [ ]:
class PCL_Model_Arch(nn.Module):
    def __init__(self,pre_trained='bert-base-uncased'):
        super().__init__()
        
        self.bert = AutoModel.from_pretrained(pre_trained, output_hidden_states=True)
        self.hidden_size = self.bert.config.hidden_size
        self.LSTM_1 = nn.LSTM(self.hidden_size,self.hidden_size,bidirectional=True)
        self.LSTM_2 = nn.LSTM(self.hidden_size,self.hidden_size,bidirectional=True)
        self.LSTM_3 = nn.LSTM(self.hidden_size,self.hidden_size,bidirectional=True)
        self.LSTM_4 = nn.LSTM(self.hidden_size,self.hidden_size,bidirectional=True)
        self.clf_1 = nn.Linear(self.hidden_size*8,30)
        self.clf_2=nn.Linear(30,30)
        self.clf_final=nn.Linear(60,2)
        
    def forward(self,text_id, text_mask,features):
        
        outputs = self.bert(input_ids=text_id,attention_mask=text_mask)
        encoded_layers = outputs[2]
        encoded_layer_1 = encoded_layers[-1].permute(1, 0, 2)
        encoded_layer_2 = encoded_layers[-2].permute(1, 0, 2)
        encoded_layer_3 = encoded_layers[-3].permute(1, 0, 2)
        encoded_layer_4 = encoded_layers[-3].permute(1, 0, 2)
        enc_hiddens, (last_hidden_1, last_cell) = self.LSTM_1(encoded_layer_1)
        enc_hiddens, (last_hidden_2, last_cell) = self.LSTM_2(encoded_layer_2)
        enc_hiddens, (last_hidden_3, last_cell) = self.LSTM_3(encoded_layer_3)
        enc_hiddens, (last_hidden_4, last_cell) = self.LSTM_4(encoded_layer_4)
        output_hidden = torch.cat((last_hidden_1[0], last_hidden_1[1],last_hidden_2[0], last_hidden_2[1],last_hidden_3[0], last_hidden_3[1],last_hidden_4[0], last_hidden_4[1]), dim=1)
        output_hidden = F.dropout(output_hidden,0.2)
        output_hidden = self.clf_1(output_hidden)
        out_2=self.clf_2(features)
        output = torch.cat((output_hidden,out_2), dim=1)
        output=self.clf_final(output)
        
        return F.softmax(output)

In [ ]:
# class PCL_Model_Arch(nn.Module):

#     def __init__(self):
#         super(PCL_Model_Arch, self).__init__()
#         self.bert = AutoModel.from_pretrained('google/canine-c', output_hidden_states=False)
#         self.drop = nn.Dropout(p=0.2)
#         self.fc = nn.Linear(768, 2)

        
#         self.softmax = nn.Softmax()

#     def forward(self, text_id, text_mask):
#         # get the last 4 layers
#         outputs= self.bert(text_id, attention_mask=text_mask)
#         # all_layers  = [4, 16, 256, 768]
#         hidden_layers = outputs[1]  # get hidden layers
#         torch.cuda.empty_cache()

        
#         x = self.drop(hidden_layers)
#         torch.cuda.empty_cache()
#         # generate logits (batch, target_size)
#         logit = self.fc(x)
#         torch.cuda.empty_cache()
#         return self.softmax(logit)

In [ ]:
tokenizer= AutoTokenizer.from_pretrained('bert-base-uncased')


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
def criterion(outputs1,  targets):

    criterion = Recall_Loss()
    loss = criterion(outputs1, targets)
    return loss

In [ ]:
class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):
    """
    Samples elements randomly from a given list of indices for imbalanced dataset
    Arguments:
        indices (list, optional): a list of indices
        num_samples (int, optional): number of samples to draw
    """

    def __init__(self, dataset, indices=None, num_samples=None):
        # if indices is not provided,
        # all elements in the dataset will be considered
        self.indices = list(range(len(dataset.label))) \
            if indices is None else indices

        # if num_samples is not provided,
        # draw `len(indices)` samples in each iteration
        self.num_samples = len(self.indices) \
            if num_samples is None else num_samples

        # distribution of classes in the dataset
        label_to_count = {}
        for idx in self.indices:
            label = self._get_label(dataset, idx)
            if label in label_to_count:
                label_to_count[label] += 1
            else:
                label_to_count[label] = 1

        # weight for each sample
        weights = [1.0 / label_to_count[self._get_label(dataset, idx)] for idx in self.indices]
        self.weights = torch.DoubleTensor(weights)

    def _get_label(self, dataset, id_):
        return dataset.label[id_]

    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(self.weights, self.num_samples, replacement=True))

    def __len__(self):
        return self.num_samples

In [ ]:
CONFIG = {"seed": 2021,
          "epochs": 3,
          "model_name": "xlnet-base-cased",
          "train_batch_size": 8,
          "valid_batch_size": 64,
          "max_length": 256,
          "learning_rate": 1e-4,
          "scheduler": 'CosineAnnealingLR',
          "min_lr": 1e-6,
          "T_max": 500,
          "weight_decay": 1e-6,
          "n_fold": 5,
          "n_accumulate": 1,
          "num_classes": 1,
          "margin": 0.5,
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          }

In [ ]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        
        text_ids = data['text_ids'].to(device, dtype = torch.long)
        text_mask = data['text_mask'].to(device, dtype = torch.long)
        fetures=data['features'].to(device, dtype = torch.float)
        targets = data['target'].to(device, dtype=torch.long)
        
        batch_size = text_ids.size(0)
        # print(targets)

        outputs = model(text_ids, text_mask,fetures)
        # print(outputs.shape)
        
        # print(outputs.shape)

        
        loss = criterion(outputs, targets)
        loss = loss / CONFIG['n_accumulate']
        loss.backward()
    
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            optimizer.step()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])
    gc.collect()
    
    return epoch_loss

In [ ]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:        
        
        text_ids = data['text_ids'].to(device, dtype = torch.long)
        text_mask = data['text_mask'].to(device, dtype = torch.long)
        targets = data['target'].to(device, dtype=torch.long)
        fetures=data['features'].to(device, dtype = torch.float)
        
        batch_size = text_ids.size(0)

        outputs = model(text_ids, text_mask,fetures)
        # outputs = outputs.argmax(dim=1)
        
        loss = criterion(outputs, targets)
        
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss,
                        LR=optimizer.param_groups[0]['lr'])   
    
    gc.collect()
    
    return epoch_loss

In [ ]:
def run_training(model, optimizer, scheduler, device, num_epochs, fold):
    # To automatically log gradients
    
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))
    
    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_loss = np.inf
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1): 
        gc.collect()
        train_epoch_loss = train_one_epoch(model, optimizer, scheduler, 
                                           dataloader=train_loader, 
                                           device=CONFIG['device'], epoch=epoch)
        
        val_epoch_loss = valid_one_epoch(model, valid_loader, device=CONFIG['device'], 
                                         epoch=epoch)
    
        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        
       
        
        # deep copy the model
        if val_epoch_loss <= best_epoch_loss:
            print(f"Validation Loss Improved ({best_epoch_loss} ---> {val_epoch_loss})")
            best_epoch_loss = val_epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            PATH = f"/content/drive/MyDrive/ISarcasm/Models/deberta_base/Loss-Fold-{fold}.bin"
            torch.save(model.state_dict(), PATH)
            # Save a model file from the current directory
            print("Model Saved")
            
        print()
    
    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best Loss: {:.4f}".format(best_epoch_loss))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [ ]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == None:
        return None
        
    return scheduler

In [ ]:
def prepare_loaders(fold):
    displacemnt_list=[0,512,1024,1536,2048,2560,3072,3584,4096,4608,4950]
    df_train = result[result.kfold != fold].reset_index(drop=True)
    df_valid = result[result.kfold == fold].reset_index(drop=True)
    sampler = ImbalancedDatasetSampler(df_train)



    
    
    train_dataset = PCLTrainDataset(df_train, tokenizer=tokenizer, max_length=CONFIG['max_length'],displacemnt=displacemnt_list[fold])
    valid_dataset = PCLTrainDataset(df_valid, tokenizer=tokenizer, max_length=CONFIG['max_length'],displacemnt=displacemnt_list[fold])

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True, drop_last=True,sampler=sampler)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

In [ ]:
result.label.unique()

array([0, 1], dtype=object)

In [ ]:
result.reset_index(inplace=True)

In [ ]:
skf = StratifiedKFold(n_splits=CONFIG['n_fold'], shuffle=True, random_state=CONFIG['seed'])

for fold, ( _, val_) in enumerate(skf.split(X=result, y=result.label.astype(int))):
    result.loc[val_ , "kfold"] = int(fold)
    
result["kfold"] = result["kfold"].astype(int)

In [ ]:
del model,train_loader, valid_loader

In [ ]:
import gc
gc.collect()

588

http://seekinginference.com/applied_nlp/bert-cnn.html

In [ ]:
for fold in range(0, CONFIG['n_fold']):
    print(f"====== Fold: {fold} ======")

    
    # Create Dataloaders
    train_loader, valid_loader = prepare_loaders(fold=fold)
    
    model = PCL_Model_Arch()
    model.to(CONFIG['device'])
    torch.cuda.empty_cache()
    
    # Define Optimizer and Scheduler
    optimizer = AdamW(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
    scheduler = fetch_scheduler(optimizer)
    
    model, history = run_training(model, optimizer, scheduler,
                                  device=CONFIG['device'],
                                  num_epochs=CONFIG['epochs'],
                                  fold=fold)
    
    
    del model, history, train_loader, valid_loader
    _ = gc.collect()
    print()

====== Fold: 0 ======


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[INFO] Using GPU: Tesla K80



100%|██████████| 9/9 [00:22<00:00,  2.53s/it, Epoch=1, LR=4.21e-5, Valid_Loss=0.23]


Validation Loss Improved (inf ---> 0.23017840114311192)
Model Saved



...

Validation Loss Improved (inf ---> 0.23056775557435377)
Model Saved



100%|██████████| 9/9 [00:22<00:00,  2.53s/it, Epoch=2, LR=3.82e-6, Valid_Loss=0.228]


Validation Loss Improved (0.23056775557435377 ---> 0.22793342237653286)
Model Saved



100%|██████████| 9/9 [00:22<00:00,  2.53s/it, Epoch=3, LR=7.46e-5, Valid_Loss=0.218]


Validation Loss Improved (0.22793342237653286 ---> 0.217538956324116)
Model Saved

Training complete in 0h 22m 30s
Best Loss: 0.2175



In [ ]:
test.dropna(inplace=True)

In [ ]:
valid_dataset = PCLTrainDataset(result_test, tokenizer=tokenizer, max_length=CONFIG['max_length'],displacemnt=0)
valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], 
                              num_workers=2, shuffle=False, pin_memory=True)

In [ ]:
@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        ids = data['text_ids'].to(device, dtype = torch.long)
        mask = data['text_mask'].to(device, dtype = torch.long)
        fetures=data['features'].to(device, dtype = torch.float)
        
        outputs = model(ids, mask,fetures)
        # outputs = outputs.argmax(dim=1)
#         print(len(outputs))
#         print(len(np.max(outputs.cpu().detach().numpy(),axis=1)))
        PREDS.append(outputs.detach().cpu().numpy()) 
    
    PREDS = np.concatenate(PREDS)
    gc.collect()
    
    return PREDS

In [ ]:
def inference(model_paths, dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = PCL_Model_Arch()
        model.to(CONFIG['device'])
        model.load_state_dict(torch.load(path))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
    
    final_preds = np.array(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    final_preds= np.argmax(final_preds,axis=1)
    return final_preds

In [ ]:
MODEL_PATH_2=['/content/drive/MyDrive/ISarcasm/Models/bert_base_cased/Loss-Fold-0.bin','/content/drive/MyDrive/ISarcasm/Models/bert_base_cased/Loss-Fold-1.bin','/content/drive/MyDrive/ISarcasm/Models/bert_base_cased/Loss-Fold-2.bin','/content/drive/MyDrive/ISarcasm/Models/bert_base_cased/Loss-Fold-3.bin','/content/drive/MyDrive/ISarcasm/Models/bert_base_cased/Loss-Fold-4.bin']
preds = inference(MODEL_PATH_2, valid_loader, CONFIG['device'])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 1


100%|██████████| 11/11 [00:28<00:00,  2.58s/it]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 2


100%|██████████| 11/11 [00:28<00:00,  2.59s/it]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

100%|██████████| 11/11 [00:28<00:00,  2.59s/it]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 4


100%|██████████| 11/11 [00:28<00:00,  2.59s/it]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Getting predictions for model 5


100%|██████████| 11/11 [00:28<00:00,  2.59s/it]


In [ ]:
from sklearn.metrics import jaccard_score,f1_score,accuracy_score,recall_score,precision_score,classification_report
def print_statistics(y, y_pred):
    accuracy = accuracy_score(y, y_pred)
    precision =precision_score(y, y_pred, average='weighted')
    recall = recall_score(y, y_pred, average='weighted')
    f_score = f1_score(y, y_pred, average='weighted')
    print('Accuracy: %.3f\nPrecision: %.3f\nRecall: %.3f\nF_score: %.3f\n'
          % (accuracy, precision, recall, f_score))
    print(classification_report(y, y_pred))
    return accuracy, precision, recall, f_score

In [ ]:
print(print_statistics(test['sarcastic'],preds))

Accuracy: 0.747
Precision: 0.732
Recall: 0.747
F_score: 0.738

              precision    recall  f1-score   support

           0       0.82      0.86      0.84       531
           1       0.45      0.37      0.41       162

    accuracy                           0.75       693
   macro avg       0.63      0.62      0.62       693
weighted avg       0.73      0.75      0.74       693

(0.7474747474747475, 0.7321282101357289, 0.7474747474747475, 0.7384186910977759)


## Prepare submission

In [ ]:
!cat task1.txt | head -n 10

In [ ]:
!cat task2.txt | head -n 10

In [ ]:
!zip submission.zip task1.txt task2.txt